In [52]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JLD2, FileIO
using JuMP

  Activating project at `~/Documents/MacroEnergy.jl`


In [53]:
include("../helper_functions.jl")

get_flows (generic function with 7 methods)

In [54]:
# Run model for no CO2 cap
(system, model) = run_case(@__DIR__; optimizer=Gurobi.Optimizer)

###### ###### ######
Running case at /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit


┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 0.19 seconds
└ @ MacroEnergy /Users/al3792/Docume

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 867269 rows, 604487 columns and 2951084 nonzeros
Model fingerprint: 0x983e723a
Coefficient statistics:
  Matrix range     [1e-03, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+06]
Presolve removed 447324 rows and 245848 columns
Presolve time: 0.93s
Presolved: 419945 rows, 358639 columns, 2021952 nonzeros
Ordering ti

┌ Info: Writing capacity results to /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit/results/capacity.csv
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/write_outputs/capacity.jl:118
┌ Info: Writing costs to /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit/results/costs.csv
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/write_outputs/costs.jl:45


(MacroEnergy.System("/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit", (ConstraintScaling = true, WriteSubcommodities = true), Dict{Symbol, DataType}(:CO2Captured => CO2Captured, :Electricity => Electricity, :CO2 => CO2, :Cement => Cement, :NaturalGas => NaturalGas, :CementMaterials => CementMaterials), Dict{Symbol, MacroEnergy.TimeData}(:Cement => MacroEnergy.TimeData{Cement}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :NaturalGas => MacroEnergy.TimeData{NaturalGas}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :CO2 => MacroEnergy.TimeData{CO2}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :Electricity => MacroEnergy.TimeData{Electricity}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :CementMaterials => MacroEnergy.TimeData{CementMaterials}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [

In [ ]:
case_path = @__DIR__
results_dir = joinpath(case_path, "results")
mkpath(results_dir)
case = "0pct_co2_reduction_capacity"

## System results
system_results_df = get_system_results_no_co2_cap(system)
MacroEnergy.write_csv(joinpath(results_dir, case * "_system_results.csv"), system_results_df)

## Capacity results
capacity_df = MacroEnergy.get_optimal_capacity(system)
MacroEnergy.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_df)

## 8760 results
flows_df = get_system_flows(system)  
MacroEnergy.write_csv(joinpath(results_dir, case * "_flows.csv"), flows_df)

"/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit/results/no_co2_cap_8760_results.csv"

In [49]:
# Run model for different CO2 caps

no_cap_co2_level = 2.295e7
co2_levels = [0.8, 0.6, 0.4, 0.2, 0]

for co2_level in co2_levels

    # Case name
    case = string(100 - floor(Int, co2_level * 100)) * "pct_co2_reduction"

    # Load system
    case_path = @__DIR__
    println("###### ###### ######")
    println("Running case at $(case_path)")
    system = MacroEnergy.load_system(case_path)

    # Set CO2 policy
    co2_policy = no_cap_co2_level * co2_level
    co2_node = MacroEnergy.get_nodes_sametype(system.locations, CO2)[1] # There is only 1 CO2 node
    co2_node.rhs_policy[CO2CapConstraint] = co2_policy
    
    # Run model
    model = MacroEnergy.generate_model(system)
    MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
    MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
    MacroEnergy.optimize!(model)

    # Save results
    ## Make results directory
    results_dir = joinpath(case_path, "results")
    mkpath(results_dir)

    ## System results
    system_results_df = get_system_results(system)
    MacroEnergy.write_csv(joinpath(results_dir, case * "_system_results.csv"), system_results_df)

    ## Capacity results
    capacity_results_df = MacroEnergy.get_optimal_capacity(system)
    MacroEnergy.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_results_df)

    ## 8760 results
    results_8760_df = get_system_flows(system)  
    MacroEnergy.write_csv(joinpath(results_dir, case * "_8760_results.csv"), results_8760_df)

    @save case * "_results.jld2" system_results_df capacity_results_df results_8760_df
    println("wrote results")
end

###### ###### ######
Running case at /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit


┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 0.18 seconds
└ @ MacroEnergy /Users/al3792/Docume

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 867269 rows, 604487 columns and 2951084 nonzeros
Model fingerprint: 0x8a09999c
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+07]
Presolve removed 447324 rows and 245848 columns
Presolve time: 0.94s
Presolved: 419945 rows, 358639 columns, 2021952 nonzeros
Ordering ti

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 0.18 seconds
└ @ MacroEnergy /Users/al3792/Docume

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 867269 rows, 604487 columns and 2951084 nonzeros
Model fingerprint: 0xdcdf4bf9
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+07]
Presolve removed 447324 rows and 245848 columns
Presolve time: 0.95s
Presolved: 419945 rows, 358639 columns, 2021952 nonzeros
Ordering ti

...

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 0.19 seconds
└ @ MacroEnergy /Users/al3792/Docume

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 867269 rows, 604487 columns and 2951084 nonzeros
Model fingerprint: 0x16051b4b
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+06]
Presolve removed 447324 rows and 245848 columns
Presolve time: 0.96s
Presolved: 419945 rows, 358639 columns, 2021952 nonzeros
Ordering ti

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/three_zones_macro_genx_cement_retrofit/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 0.19 seconds
└ @ MacroEnergy /Users/al3792/Docume

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 867269 rows, 604487 columns and 2951084 nonzeros
Model fingerprint: 0x4044060a
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 377245 rows and 272131 columns
Presolve time: 0.07s

Barrier solved model in 0 iterations and 0.07 seconds (0.22 work un

MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [55]:
MacroEnergy.get_optimal_flow(system)

Row,commodity,commodity_subtype,zone,resource_id,component_id,type,variable,segment,time,value
,Symbol,Symbol,Symbol,Symbol,Symbol,Symbol,Symbol,Int64,Int64,Float64
1,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,1,20.6051
2,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,2,20.5979
3,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,3,20.5946
4,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,4,20.5925
5,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,5,20.5983
6,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,6,20.6074
7,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,7,20.6088
8,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,8,20.6014
9,CementMaterials,flow,cement_input_materials_source,cement_MA,cement_MA_cement_materials_edge,MacroEnergy.ElectrochemCementPlant{CementMaterials},flow,1,9,20.5943
